In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hmadarw\\Learnings\\DataScience\\KrishNaik\\Project\\datascienceproject\\experiments'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hmadarw\\Learnings\\DataScience\\KrishNaik\\Project\\datascienceproject'

In [5]:
from dataclasses import dataclass # no need to provide constructor
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [18]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

In [6]:
from src.data_science_project.constants import *
from src.data_science_project.utils.helper import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
from src.data_science_project import logger
import zipfile

In [9]:
# component-Data Ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-04 15:36:08,274]: INFO: helper: Yaml file: config\config.yaml loaded successfully]
[2026-01-04 15:36:08,277]: INFO: helper: Yaml file: params.yaml loaded successfully]


[2026-01-04 15:36:08,285]: INFO: helper: Yaml file: schema.yaml loaded successfully]
[2026-01-04 15:36:08,293]: INFO: helper: created directory at: artifacts]
[2026-01-04 15:36:08,293]: INFO: helper: created directory at: artifacts/data_ingestion]
[2026-01-04 15:36:09,248]: INFO: 242576026: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B778:3158B8:177D402:3132EAC:695A2FA8
Accept-Ranges: bytes
Date: Sun, 04 Jan 2026 10:06:08 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210027-BOM
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1767521168.267876,VS0,VE210
Vary: Authorization,Acce